In [1]:
import requests
import pandas

In [2]:
API_KEY = "881078-ec5446d8-7fbd-4fac-806d-8a4d81eece36"
URL = "https://api.agendor.com.br/v3/deals"


headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}

In [9]:
req = requests.get('https://api.agendor.com.br/v3/deal_stages', headers=headers)

In [5]:
# Função para realizar o scraping
def fetch_data():
    next_url = URL
    params = {"per_page": 100}
    all_json_deals_data = []
    wait_time = 10  # Tempo inicial de espera em segundos

    while next_url:
        try:
            response = requests.get(next_url, headers=headers, params=params)

            if response.status_code == 200:
                json_deal_data = response.json().get('data', [])
                all_json_deals_data.extend(json_deal_data)
                links = response.json().get('links', {})
                next_url = links.get('next', False)
                wait_time = 10  # Reseta o tempo de espera se der certo

            elif response.status_code == 429:
                print(f"Erro 429: Muitas requisições. Aguardando {wait_time} segundos antes de tentar novamente...")
                time.sleep(wait_time)
                wait_time *= 2  # Dobra o tempo de espera a cada falha

            else:
                print(f"Erro {response.status_code}: {response.text}")
                break  # Sai do loop se for outro erro

        except requests.exceptions.JSONDecodeError:
            print("Erro ao decodificar JSON. Resposta vazia ou malformada.")
            break
        except requests.exceptions.RequestException as e:
            print(f"Erro na requisição: {e}")
            break

    return all_json_deals_data

In [7]:
data = fetch_data()

In [68]:
ids = []
for deal in data:
    if deal['dealStage']['name'] == '1.3 DIAGNÓSTICO' and deal['dealStatus']['name'] == 'Em andamento':
        ids.append(deal['id']) 

In [72]:
deal

{'id': 27095653,
 'parentDealId': None,
 'accountId': 698720,
 'organization': None,
 'person': None,
 'author': {'accountId': 698720, 'id': 860639, 'name': 'Guilherme Hely'},
 'dealStage': {'id': 3182353,
  'name': '1.1 LEADS',
  'sequence': 1,
  'funnel': {'id': 772312, 'name': '1 POS'}},
 'dealStatus': {'id': 1, 'name': 'Em andamento'},
 'lossReason': None,
 'owner': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
 'title': '227 - Edivan',
 'description': None,
 'startTime': '2024-10-24T00:00:00.000Z',
 'endTime': None,
 'wonAt': None,
 'lostAt': None,
 'createdAt': '2024-10-24T20:46:27.000Z',
 'updatedAt': '2024-11-08T14:42:10.000Z',
 'ranking': 0,
 'value': 0.0,
 'products': [],
 'allowedUsers': [{'accountId': 698720,
   'id': 860639,
   'name': 'Guilherme Hely'},
  {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
  {'accountId': 698720, 'id': 860980, 'name': 'Gustavo Porto'},
  {'accountId': 698720, 'id': 863021, 'name': 'Juliane Sales'},
  {'accountId': 

In [173]:
leads_id = []
for deal in data:
    if deal['dealStage']['name'] == '1.3 DIAGNÓSTICO' and deal['person'] and deal['dealStatus']['name'] == 'Em andamento':
        leads_id.append(deal['person']['id'])

In [177]:
len(leads_id)

157

In [76]:
def agendor_request(url, headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}, params=None, wait_time = 10):
    base_url = 'https://api.agendor.com.br/v3/'
    full_url = base_url + url
    while True:
        response = requests.get(full_url, headers=headers, params=params)
        if response.status_code == 200:
            return response
        elif response.status_code == 429:
            print(f"Erro 429: Muitas requisições. Aguardando {wait_time} segundos antes de tentar novamente...")
            time.sleep(wait_time)
            wait_time *= 2  # Dobra o tempo de espera a cada falha
        else:
            return None

In [36]:
URL = 'https://api.agendor.com.br/v3/people'

# Função para realizar o scraping
def fetch_data():
    next_url = URL
    params = {"per_page": 100}
    all_json_deals_data = []
    wait_time = 10  # Tempo inicial de espera em segundos

    while next_url:
        try:
            response = requests.get(next_url, headers=headers, params=params)

            if response.status_code == 200:
                json_deal_data = response.json().get('data', [])
                all_json_deals_data.extend(json_deal_data)
                links = response.json().get('links', {})
                next_url = links.get('next', False)
                wait_time = 10  # Reseta o tempo de espera se der certo

            elif response.status_code == 429:
                print(f"Erro 429: Muitas requisições. Aguardando {wait_time} segundos antes de tentar novamente...")
                time.sleep(wait_time)
                wait_time *= 2  # Dobra o tempo de espera a cada falha

            else:
                print(f"Erro {response.status_code}: {response.text}")
                break  # Sai do loop se for outro erro

        except requests.exceptions.JSONDecodeError:
            print("Erro ao decodificar JSON. Resposta vazia ou malformada.")
            break
        except requests.exceptions.RequestException as e:
            print(f"Erro na requisição: {e}")
            break

    return all_json_deals_data

In [38]:
person_data = fetch_data()

In [179]:
leads_data = []
for person in person_data:
    if person['id'] in leads_id:
        leads_data.append(person)

In [181]:
import re

In [183]:
bad_leads = []
good_leads = []
for lead_data in leads_data:
    number = lead_data['contact']['work']
    if number:
        ddd = re.search(r'\((\d+)\)', number).group(1)
        ddd = int(ddd)
        if ddd < 30 or ddd > 40:
            bad_leads.append(lead_data)
        else: 
            good_leads.append(lead_data)
    else:
        bad_leads.append(lead_data)
        

In [189]:
for person_id in bad_leads:
    https://api.agendor.com.br/v3/people/{id}

80

In [84]:
good_leads_id = [good_lead['id'] for good_lead in good_leads]

In [86]:
len(good_leads_id)
len(bad_leads_id)

NameError: name 'bad_leads_id' is not defined

In [88]:
good_deal_ids = []
for deal in data:
    person = deal['person']
    if person and deal['dealStage']['name'] == '1.3 DIAGNÓSTICO' and deal['dealStatus']['name'] == 'Em andamento':
        if person['id'] in good_leads_id:
            good_deal_ids.append(deal['id'])

In [90]:
bad_deals_id = []
for deal in data:
    if deal['dealStage']['name'] == '1.3 DIAGNÓSTICO' and deal['id'] not in good_deal_ids and deal['dealStatus']['name'] == 'Em andamento':
        bad_deals_id.append(deal['id'])

In [98]:
len(good_deal_ids) + len(bad_deals_id)

157

In [213]:
import time
payload = {"dealStatusText": "lost"}  # Atualizando para "lost"
BASE_URL = "https://api.agendor.com.br/v3"

i = 0
for deal_id in bad_deals_id[88:]:
    url = f"{BASE_URL}/deals/{deal_id}/status"
    wait_time = 10  # Tempo de espera inicial

    while True:
        response = requests.put(url, headers=headers, json=payload)

        if response.status_code == 200:
            wait_time = 10  # Reseta o tempo de espera se der certo
            print(f"Sucesso para {i}")  # Corrigido o uso de f-string
            break
            
        elif response.status_code == 429:
            print(f"Erro 429: Muitas requisições. Aguardando {wait_time} segundos antes de tentar novamente...")
            time.sleep(wait_time)
            wait_time *= 2  # Dobra o tempo de espera a cada falha
        else:
            print(f"Erro inesperado para {i}: {response.status_code}")
            break  # Você pode optar por parar ou continuar com outro comportamento

    i += 1
    print(f"Processado {i} de {len(bad_deals_id)}")

Sucesso para 0
Processado 1 de 119
Sucesso para 1
Processado 2 de 119
Sucesso para 2
Processado 3 de 119
Sucesso para 3
Processado 4 de 119
Sucesso para 4
Processado 5 de 119
Sucesso para 5
Processado 6 de 119
Sucesso para 6
Processado 7 de 119
Sucesso para 7
Processado 8 de 119
Sucesso para 8
Processado 9 de 119
Sucesso para 9
Processado 10 de 119
Sucesso para 10
Processado 11 de 119
Sucesso para 11
Processado 12 de 119
Sucesso para 12
Processado 13 de 119
Sucesso para 13
Processado 14 de 119
Sucesso para 14
Processado 15 de 119
Sucesso para 15
Processado 16 de 119
Sucesso para 16
Processado 17 de 119
Sucesso para 17
Processado 18 de 119
Sucesso para 18
Processado 19 de 119
Sucesso para 19
Processado 20 de 119
Sucesso para 20
Processado 21 de 119
Sucesso para 21
Processado 22 de 119
Sucesso para 22
Processado 23 de 119
Sucesso para 23
Processado 24 de 119
Sucesso para 24
Processado 25 de 119
Sucesso para 25
Processado 26 de 119
Sucesso para 26
Processado 27 de 119
Sucesso para 27
Proc

In [201]:
bad_leads_id = [bad_lead['id'] for bad_lead in bad_leads]

In [205]:
len(bad_leads_id)

61

In [209]:
#payload = {"dealStatusText": "lost"}  # Atualizando para "lost"
BASE_URL = "https://api.agendor.com.br/v3"

i = 0
for lead_id in bad_leads_id:
    url = f"{BASE_URL}/people/{lead_id}"
    wait_time = 10  # Tempo de espera inicial

    while True:
        response = requests.delete(url, headers=headers)

        if response.status_code == 200:
            wait_time = 10  # Reseta o tempo de espera se der certo
            print(f"Sucesso para {i}")  # Corrigido o uso de f-string
            break
            
        elif response.status_code == 429:
            print(f"Erro 429: Muitas requisições. Aguardando {wait_time} segundos antes de tentar novamente...")
            time.sleep(wait_time)
            wait_time *= 2  # Dobra o tempo de espera a cada falha
        else:
            print(f"Erro inesperado para {i}: {response.status_code}")
            break  # Você pode optar por parar ou continuar com outro comportamento

    i += 1
    print(f"Processado {i} de {len(bad_leads_id)}")

Sucesso para 0
Processado 1 de 119
Sucesso para 1
Processado 2 de 119
Sucesso para 2
Processado 3 de 119
Sucesso para 3
Processado 4 de 119
Sucesso para 4
Processado 5 de 119
Sucesso para 5
Processado 6 de 119
Sucesso para 6
Processado 7 de 119
Sucesso para 7
Processado 8 de 119
Sucesso para 8
Processado 9 de 119
Sucesso para 9
Processado 10 de 119
Sucesso para 10
Processado 11 de 119
Sucesso para 11
Processado 12 de 119
Sucesso para 12
Processado 13 de 119
Sucesso para 13
Processado 14 de 119
Sucesso para 14
Processado 15 de 119
Sucesso para 15
Processado 16 de 119
Sucesso para 16
Processado 17 de 119
Sucesso para 17
Processado 18 de 119
Sucesso para 18
Processado 19 de 119
Sucesso para 19
Processado 20 de 119
Sucesso para 20
Processado 21 de 119
Sucesso para 21
Processado 22 de 119
Sucesso para 22
Processado 23 de 119
Sucesso para 23
Processado 24 de 119
Sucesso para 24
Processado 25 de 119
Sucesso para 25
Processado 26 de 119
Sucesso para 26
Processado 27 de 119
Sucesso para 27
Proc

In [195]:
requests.del

SyntaxError: invalid syntax (2214764233.py, line 1)

In [143]:
deal_id = bad_deals_id[49]
url = f"{BASE_URL}/deals/{deal_id}/status"
response = requests.put(url, headers=headers, json=payload)
response.json()

{'errors': ['Title There can only be one deal with this title for this organization/person']}

In [153]:
len(bad_leads)

196

In [139]:
len(bad_deals_id)

119